In [4]:
import pickle
import json
import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import scipy.special as spcl

import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as psb
import plotly.io as pio

import analysis_module as anlyz

pio.templates.default = "none"

# %load_ext autoreload
# %autoreload 2

%matplotlib widget

In [5]:
station_list = anlyz.station_list(exclude="8638901")
station_datasets = []

pctl = ["10", "50", "90"]
scenario_keys = ["low", "int_low", "int", "int_high", "high"]

for sta_n, station in station_list.iterrows():

    if station["id"] not in ["1617433", "1617760", "1612340"]:
        continue

    print(f"{station['id']}: {station['tool_name']}")

    meta_arrays = dict()

    meta_arrays["station_id"] = xr.DataArray(
        data=[station["id"]],
        dims=["station"],
        coords=dict(station=("station", [sta_n])),
        attrs=dict(description="unique NOAA station identification number"),
    )

    meta_arrays["station_name"] = xr.DataArray(
        data=[station["tool_name"]],
        dims=["station"],
        coords=dict(station=("station", [sta_n])),
        attrs=dict(description="name of station location"),
    )

    meta_arrays["longitude"] = xr.DataArray(
        data=[station["lon"]],
        dims=["station"],
        coords=dict(station=("station", [sta_n])),
        attrs=dict(description="longitude of station", units="degrees east"),
    )

    meta_arrays["latitude"] = xr.DataArray(
        data=[station["lat"]],
        dims=["station"],
        coords=dict(station=("station", [sta_n])),
        attrs=dict(description="latitude of station", units="degrees north"),
    )

    sl = xr.open_dataset(f"../data/tide_gauge/{station.id}.nc")
    sl = sl.observed.to_pandas().loc["2001":"2020"]*100 # cm
    dymx = sl.groupby(pd.Grouper(freq="D")).apply(
        lambda x: x.max() if x.count() == 24 else None
    )
    target_count = dymx.count()/365.25 * 5
    for lev in range(500):
        if (dymx >= lev).sum() > target_count:
            continue
        else:
            lev_kt = lev - 1
            break

    meta_arrays["level_kt_0120"] = xr.DataArray(
        data=[lev_kt],
        dims=["station"],
        coords=dict(station=("station", [sta_n])),
        attrs=dict(description="highest level exceeded at least 5 times on average during 2001–2020", units="centimeters above MHHW"),
    )

    meta_dataset = xr.Dataset(meta_arrays)

    sta_path = f"./ensemble_stats/{station['id']}/"

    scenario_datasets = []
    for scenario in scenario_keys:

        files = glob.glob(f"{sta_path}{scenario}/*")

        a = {p: dict() for p in pctl}
        for fn in files:

            with open(fn, "r") as f:
                d = json.load(f)

            thrsh = fn[-8:-5]

            for p in pctl:
                yrs = d["annual_percentiles"]["years"]
                xd = d["annual_percentiles"]["percentiles"][p]
                a[p][thrsh] = pd.Series(xd, index=yrs)

        df = {p: pd.DataFrame(a[p]).sort_index(axis=1, ascending=False) for p in pctl}

        lev_kt = (df["50"] >= 5).idxmax(axis=1)
        years = lev_kt.index.tolist()
        variables = dict()
        variables["level_kt"] = {
            "values": [int(lev) for lev in lev_kt.tolist()],
            "description": "highest level exceeded at least 5 times on average for each year",
            "units": "centimeters above MHHW",
        }
        for p in ["10", "50", "90"]:
            variables[f"flood_days_p{p}"] = {
                "values": [df[p].loc[y, h] for y, h in zip(years, lev_kt.values)],
                "description": f"{p}th percentile of flooding days for level_kt",
                "units": "days per year",
            }

        data_arrays = dict()
        for v in variables:
            data_arrays[v] = xr.DataArray(
                data=[[variables[v]["values"]]],
                dims=["station", "scenario", "year"],
                coords=dict(
                    station=("station", [sta_n]),
                    scenario=("scenario", [scenario]),
                    year=("year", years),
                ),
                attrs=dict(
                    description=variables[v]["description"],
                    units=variables[v]["units"],
                ),
            )

        scenario_datasets.append(xr.Dataset(data_arrays))

    station_datasets.append(
        xr.merge([meta_dataset, xr.concat(scenario_datasets, dim="scenario")])
    )

puho_analysis = xr.concat(station_datasets, dim="station")

puho_analysis["scenario_names"] = xr.DataArray(
    data=["Low", "Intermediate Low", "Intermediate", "Intermediate High", "High"],
    dims=["scenario"],
    coords=dict(scenario=("scenario", scenario_keys)),
    attrs=dict(description="Names of the U.S. Interagency SLR scenarios"),
)

puho_analysis.to_netcdf("./puho_analysis.nc")

1612340: Honolulu, HI
1617433: Kawaihae, HI
1617760: Hilo, HI


In [6]:
puho_analysis

<xarray.Dataset>
Dimensions:         (station: 3, scenario: 5, year: 81)
Coordinates:
  * station         (station) int64 11 14 15
  * scenario        (scenario) <U8 'low' 'int_low' 'int' 'int_high' 'high'
  * year            (year) int64 2020 2021 2022 2023 ... 2097 2098 2099 2100
Data variables:
    station_id      (station) <U7 '1612340' '1617433' '1617760'
    station_name    (station) <U12 'Honolulu, HI' 'Kawaihae, HI' 'Hilo, HI'
    longitude       (station) float64 202.1 204.2 204.9
    latitude        (station) float64 21.31 20.04 19.73
    level_kt_0120   (station) int64 31 32 33
    level_kt        (station, scenario, year) int64 40 36 37 36 ... 271 276 282
    flood_days_p10  (station, scenario, year) int64 1 1 1 1 1 0 ... 1 1 0 1 1 1
    flood_days_p50  (station, scenario, year) int64 5 6 6 5 6 5 ... 6 6 5 6 5 5
    flood_days_p90  (station, scenario, year) int64 14 20 21 18 ... 16 18 16 14
    scenario_names  (scenario) <U17 'Low' 'Intermediate Low' ... 'High'